In [1]:
%pip install -r requirements.txt

import kagglehub
from kagglehub import KaggleDatasetAdapter

def init_data_set():
    # Set the path to the file you'd like to load
    file_path = "loan.csv"
    
    # Load the latest version
    df = kagglehub.load_dataset(
      KaggleDatasetAdapter.PANDAS,
      "adarshsng/lending-club-loan-data-csv",
      file_path,
      # Provide any additional arguments like 
      # sql_query or pandas_kwargs. See the 
      # documenation for more information:
      # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
    )
    
    print("First 5 records:", df.head())

  Using cached kagglehub-1.0.0-py3-none-any.whl.metadata (40 kB)
  Using cached matplotlib-3.10.8-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (52 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached kagglesdk-0.1.15-py3-none-any.whl.metadata (13 kB)
  Using cached contourpy-1.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.61.1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (114 kB)
  Using cached kiwisolver-1.4.9-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.3 kB)
  Using cached pillow-12.1.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (8.8 kB)
Using cached kagglehub-1.0.0-py3-none-any.whl (70 kB)
Using cached kagglesdk-0.1.15-py3-none-any.whl (160 kB)
Using cached matplotlib-3.10.8-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.w

As an investor, the $RecoveryRatio$ is among the most important considerations, more than whether or not a loan defaults. The $RecoveryRation > 1$ (sans fees) means that the investor has made money on the loan.

$$
RecoveryRatio = \frac{total\_pymnt}{loan\_amnt}
$$

An investor can make money on a loan even if it defaults

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import kagglehub

init_data_set()

def show_default_loans():     
    df['recovery_ratio'] = df['total_pymnt'] / df['loan_amnt']
    
    charged_off_loans = df[df['loan_status'] == 'Charged Off'].copy()

show_default_loans()

/tmp/ipykernel_325/3566176672.py:11: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(
